In [1]:
import pandas as pd
from sklearn import preprocessing

In [2]:
data = pd.read_csv("../outputs/clean_houses.csv", index_col=0)

In [3]:
data.head()

,neighborhood_id,house_type_id,is_renewal_needed,is_new_development,sq_mt_built,n_rooms,n_bathrooms,floor,is_exterior,has_lift,has_parking,has_storage_room,has_terrace,has_balcony,has_pool,buy_price,buy_price_by_area,latitude,longitude
0,"San Cristóbal, Villaverde(Madrid)",HouseType 1: Pisos,False,False,64.0,2,1.0,3,True,False,False,False,False,False,False,85000,1328,40.343263,-3.688352
1,"Los Ángeles, Villaverde(Madrid)",HouseType 1: Pisos,True,False,70.0,3,1.0,4,True,True,False,False,True,False,False,129900,1856,40.355109,-3.700105
2,"San Andrés, Villaverde(Madrid)",HouseType 1: Pisos,False,False,94.0,2,2.0,1,True,True,False,True,False,False,False,144247,1535,40.345539,-3.710970
3,"San Andrés, Villaverde(Madrid)",HouseType 1: Pisos,False,False,64.0,2,1.0,Bajo,True,True,False,True,False,False,False,109900,1717,40.345539,-3.710970
4,"Los Rosales, Villaverde(Madrid)",HouseType 1: Pisos,False,False,108.0,2,2.0,4,True,True,True,True,False,False,True,260000,2407,40.355841,-3.688399


In [4]:
data = data.rename(columns={"sq_mt_built": "m2"})

## Para realizar nuestra predicción necesitamos que todos nuestros datos sean numéricos. Vamos a comprobar por tanto, que tipos de datos tenemos y realizar un pequeño análisis de los mismos.

In [5]:
# Para realizar nuestra predicción necesitamos que todos nuestros datos sean numéricos.
# Vamos a comprobar por tanto que tipos de datos tenemos.

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17398 entries, 0 to 17397
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   neighborhood_id     17398 non-null  object 
 1   house_type_id       17398 non-null  object 
 2   is_renewal_needed   17398 non-null  bool   
 3   is_new_development  17398 non-null  bool   
 4   m2                  17398 non-null  float64
 5   n_rooms             17398 non-null  int64  
 6   n_bathrooms         17398 non-null  float64
 7   floor               17398 non-null  object 
 8   is_exterior         17398 non-null  bool   
 9   has_lift            17398 non-null  bool   
 10  has_parking         17398 non-null  bool   
 11  has_storage_room    17398 non-null  bool   
 12  has_terrace         17398 non-null  bool   
 13  has_balcony         17398 non-null  bool   
 14  has_pool            17398 non-null  bool   
 15  buy_price           17398 non-null  int64  
 16  buy_

In [6]:
# Realizamos también un análisis estadístico.

data.describe()

,m2,n_rooms,n_bathrooms,buy_price,buy_price_by_area,latitude,longitude
count,17398.000000,17398.000000,17398.000000,1.739800e+04,17398.000000,17398.000000,17398.000000
mean,124.480400,2.866536,1.873204,5.660945e+05,4107.643407,40.423783,-3.688985
std,86.350814,1.230238,1.033226,6.148399e+05,1929.275478,0.035425,0.036761
min,16.000000,0.000000,1.000000,3.600000e+04,447.000000,40.339934,-3.828783
25%,70.000000,2.000000,1.000000,1.989685e+05,2617.250000,40.397292,-3.708916
50%,97.000000,3.000000,2.000000,3.600000e+05,3858.000000,40.426708,-3.693014
75%,146.000000,3.000000,2.000000,6.750000e+05,5114.000000,40.449115,-3.667970
max,1165.000000,15.000000,14.000000,8.800000e+06,18889.000000,40.520554,-3.546616


In [7]:
# Vamos a crear una función que transforme los datos numéricos

def transf(column):

    le = preprocessing.LabelEncoder()
    data[column] = le.fit_transform(data[column])
    return data[column]

In [8]:
# Transformamos ahora todas las columnas type object y bool a float64, excepto floor.
# A la columna floor le daremos un tratamiento previo.

transf("house_type_id")
transf("is_renewal_needed")
transf("is_new_development")
transf("is_exterior")
transf("has_lift")
transf("has_parking")
transf("has_storage_room")
transf("has_terrace")
transf("has_balcony")
transf("has_pool");

In [9]:
# Tratamos ahora los datos de la columna floor.

data.floor.value_counts()

1                       4046
2                       3233
3                       2722
4                       2117
Bajo                    1829
5                       1225
6                        844
7                        537
8                        309
Entreplanta exterior     225
9                        175
Semi-sótano exterior      50
Entreplanta interior      32
Semi-sótano interior      29
Sótano interior           21
Sótano exterior            4
Name: floor, dtype: int64

In [10]:
# Creamos un diccionario para aquellas plantas que figuran como texto.

planta={"Entreplanta exterior": 0.5,
        "Entreplanta interior": 0.5,
        "Bajo": 0,
        "Semi-sótano exterior": -1,    
        "Semi-sótano interior": -1,   
        "Sótano interior": -2,          
        "Sótano exterior": -2}

In [11]:
# Creamos una función que cambie los valores de la columna floor a partir del diccionario creado.
# Transformamos la columna en datos numéricos.

data.floor = data.floor.apply(lambda x: planta.get(x, x))
data.floor = pd.to_numeric(data.floor)

In [12]:
# Vamos a seleccionar las columnas con las que finalmente trabajaremos.
# No vamos a seleccionar la columna neighborhood_id, ya que tenemos latitud y longitud.
# No vamos a seleccionar la columna buy_price_by_area para evitar que nuestro errores en nuestro modelo.
# En este caso al tener las columnas m2 y buy_price_by_area, el modelo sería casi perfecto.

data = data [["house_type_id", "is_renewal_needed", "is_new_development",
              "m2", "n_rooms", "n_bathrooms", "floor", "is_exterior",
              "has_lift", "has_parking", "has_storage_room", "has_terrace",
              "has_balcony", "has_pool", "latitude", "longitude", "buy_price"]]

In [13]:
# Vamos a echar un vistazo a nuestra tabla definitiva

data.head()

,house_type_id,is_renewal_needed,is_new_development,m2,n_rooms,n_bathrooms,floor,is_exterior,has_lift,has_parking,has_storage_room,has_terrace,has_balcony,has_pool,latitude,longitude,buy_price
0,0,0,0,64.0,2,1.0,3.0,1,0,0,0,0,0,0,40.343263,-3.688352,85000
1,0,1,0,70.0,3,1.0,4.0,1,1,0,0,1,0,0,40.355109,-3.700105,129900
2,0,0,0,94.0,2,2.0,1.0,1,1,0,1,0,0,0,40.345539,-3.710970,144247
3,0,0,0,64.0,2,1.0,0.0,1,1,0,1,0,0,0,40.345539,-3.710970,109900
4,0,0,0,108.0,2,2.0,4.0,1,1,1,1,0,0,1,40.355841,-3.688399,260000


In [14]:
data.to_csv("../outputs/house_fit.csv")